# --- Day 8: Space Image Format ---
The Elves' spirits are lifted when they realize you have an opportunity to reboot one of their Mars rovers, and so they are curious if you would spend a brief sojourn on Mars. You land your ship near the rover.

When you reach the rover, you discover that it's already in the process of rebooting! It's just waiting for someone to enter a BIOS password. The Elf responsible for the rover takes a picture of the password (your puzzle input) and sends it to you via the Digital Sending Network.

Unfortunately, images sent via the Digital Sending Network aren't encoded with any normal encoding; instead, they're encoded in a special Space Image Format. None of the Elves seem to remember why this is the case. They send you the instructions to decode it.

Images are sent as a series of digits that each represent the color of a single pixel. The digits fill each row of the image left-to-right, then move downward to the next row, filling rows top-to-bottom until every pixel of the image is filled.

Each image actually consists of a series of identically-sized layers that are filled in this way. So, the first digit corresponds to the top-left pixel of the first layer, the second digit corresponds to the pixel to the right of that on the same layer, and so on until the last digit, which corresponds to the bottom-right pixel of the last layer.

In [21]:
import numpy as np
from collections import Counter

width = 25
height = 6

test1 = "123456789012"

def str_to_array(pixels, width, height):
    ar = np.array([int(x) for x in pixels])
    layers = len(ar) / (width * height)
    return ar.reshape([-1, height, width])

str_to_array(test1, 3, 2)

array([[[1, 2, 3],
        [4, 5, 6]],

       [[7, 8, 9],
        [0, 1, 2]]])

In [14]:
with open('../data/08a.txt', 'r') as f:
    pixels = f.read().strip()
len(pixels)

15000

In [37]:
layered_pixels = str_to_array(pixels, width, height)

counts = {}
for i, layer in enumerate(layered_pixels):
    c = Counter(layer.reshape(-1,))
    counts[i] = c

layer_fewest_zeros = min(counts, key=lambda x: counts[x][0])
counts[layer_fewest_zeros]

Counter({2: 129, 1: 16, 0: 5})

In [35]:
counts[11][1] * counts[11][2]

2064

In [141]:
shaped_layered = layered_pixels.reshape(1,-1,len(layered_pixels))[0]
shaped_layered

array([[2, 2, 2, ..., 2, 1, 2],
       [2, 0, 2, ..., 2, 2, 2],
       [0, 0, 2, ..., 2, 2, 1],
       ...,
       [2, 2, 2, ..., 1, 1, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 2, 1, ..., 0, 1, 1]])

In [139]:
test2 = str_to_array('0222112222120000', 2, 2)

shaped = test2.reshape(1,-1,4)[0]
shaped

array([[0, 2, 2, 2],
       [1, 1, 2, 2],
       [2, 2, 1, 2],
       [0, 0, 0, 0]])

In [133]:

#print('output', output)


def combine_layers(shaped):
    output = np.array([2 for _ in range(len(shaped))])
    
    for layer in shaped[::-1]:
        new_output = []
        
        for new, old in zip(layer, output[:]):
            if new != 2:
                new_output.append(new)
            else: 
                new_output.append(old)

        output = np.array(new_output)
    return output
    
            
combine_layers(shaped)      

array([0, 1, 1, 0])

In [143]:
combined = combine_layers(shaped_layered)
combined.reshape(width, height)

ValueError: cannot reshape array of size 100 into shape (25,6)